# 5. Catalyst + Oscar

**TODO**
- Write motivation
- Getting CRN equations at steady state
- (can solve for steady state equations also?)
- CRN -> GB simple example
- Cholesterol example
- Independent decomposition

In [174]:
using Pkg
Pkg.activate(".")
Pkg.add("Symbolics")

  Activating project at `~/Documents/repos/JuliaTutorials`
   Resolving package versions...
  No Changes to `~/Documents/repos/JuliaTutorials/Project.toml`
  No Changes to `~/Documents/repos/JuliaTutorials/Manifest.toml`


In [175]:
using Symbolics
using Catalyst
using Catalyst: unknowns # sometimes need to specify which method to use if defined in multiple packages
using Oscar
using Oscar: groebner_basis

### Aside: Symbolics.jl
Catalyst uses `Symbolics.jl` as its computer algebra system. This is slightly different to Oscar.

In [176]:
@variables x y z

3-element Vector{Num}:
 x
 y
 z

In [177]:
x isa Number  # true

f(t) = t^2 + t
sym_vec = f.([2x, y^2 + 1, x + z + y])

3-element Vector{Num}:
         2x + (4//1)*(x^2)
     1 + y^2 + (1 + y^2)^2
 x + y + z + (x + y + z)^2

In [178]:
Symbolics.expand.(sym_vec)

3-element Vector{Num}:
                          (2//1)*x + (4//1)*(x^2)
                                 2 + 3(y^2) + y^4
 x + y + z + x^2 + 2x*y + 2x*z + y^2 + 2y*z + z^2

## Catalyst to Oscar

In [181]:
rn = @reaction_network begin
    k, 2A + 3B --> A + 2C + D
    b, C + D --> 2A + 3B
end

Model ##ReactionSystem#302:
Unknowns (4): see unknowns(##ReactionSystem#302)
  A(t)
  B(t)
  C(t)
  D(t)
Parameters (2): see parameters(##ReactionSystem#302)
  k
  b

In [182]:
odesys = convert(ODESystem, rn, combinatoric_ratelaws=false)

Model ##ReactionSystem#302:
Equations (4):
  4 standard: see equations(##ReactionSystem#302)
Unknowns (4): see unknowns(##ReactionSystem#302)
  A(t)
  B(t)
  C(t)
  D(t)
Parameters (2): see parameters(##ReactionSystem#302)
  k
  b

We can extract the mass-action polynomials from the ODESystem struct.

In [183]:
@show vars = unknowns(odesys)
@show params = parameters(odesys)
eqn_rhs = [eq.rhs for eq in equations(odesys)]

vars = unknowns(odesys) = SymbolicUtils.BasicSymbolic{Real}[A(t), B(t), C(t), D(t)]
params = parameters(odesys) = SymbolicUtils.BasicSymbolic{Real}[k, b]


4-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 2b*D(t)*C(t) - k*(B(t)^3)*(A(t)^2)
 3b*D(t)*C(t) - 3k*(B(t)^3)*(A(t)^2)
 -b*D(t)*C(t) + 2k*(B(t)^3)*(A(t)^2)
 -b*D(t)*C(t) + k*(B(t)^3)*(A(t)^2)

We first need to convert between the symbolic representations of Catalyst and Oscar (this is the most fiddly part).

In [184]:
function polynomial_ideal(eqn_rhs, vars, params)
    # Convert Catalyst symbolic variables into Julia Symbol types
    varnames = tosymbol.(vars, escape=false)
    paramnames = tosymbol.(params)
    
    # Create polynomial ring in Oscar
    CC, oscar_coeffs = polynomial_ring(QQ, paramnames)
    ff = fraction_field(CC)
    RR, oscar_vars = polynomial_ring(ff, varnames)
    
    # Map Catalyst variables to Oscar variables
    cat_var_params = [vars; params]
    oscar_var_params = [oscar_vars; oscar_coeffs]
    cat_to_oscar = Dict(cat => oscar for (cat, oscar) in zip(cat_var_params, oscar_var_params))
    oscar_to_cat = Dict((oscar => cat) for (cat, oscar) in cat_to_oscar)
    
    # build Oscar polynomial by substituting oscar vars in Catalyst equations RHS (Right Hand Sides)
    polys = map(eqn_rhs) do rhs
        if rhs isa Number  # is a constant e.g. zero
            RR(rhs)
        else
            Symbolics.substitute(rhs, cat_to_oscar)
        end
    end
    ideal(RR, polys)
end

I = polynomial_ideal(eqn_rhs, vars, params)

Ideal generated by
  -k*A^2*B^3 + 2*b*C*D
  -3*k*A^2*B^3 + 3*b*C*D
  2*k*A^2*B^3 - b*C*D
  k*A^2*B^3 - b*C*D

Now we can find a Groebner basis for $I$

In [185]:
G = groebner_basis(I)

Gröbner basis with elements
  1: C*D
  2: k*A^2*B^3 - b*C*D
with respect to the ordering
  degrevlex([A, B, C, D])

In [186]:
# we may want to map back to Catalyst/Symbolics
function to_symbolic_polynomial(poly::MPolyRingElem)
    # create new symbolic vars from ring of poly
    var_syms = Symbolics.variable.(gens(parent(poly)))
    coeff_syms = Symbolics.variable.(gens(coefficient_ring(poly)))

    # coefficients
    coeff_terms = Oscar.coefficients(poly)
    coeffs = zeros(Num, length(coeff_terms))
    for (i, term) in enumerate(coeff_terms)
        cf_and_es = coefficients_and_exponents(term.num)
        coeff_polyn = sum(Int(cf) * prod(coeff_syms .^ es) for (cf, es) in cf_and_es)
        coeffs[i] = coeff_polyn
    end
    # polynomial variables
    exp_vecs = collect(Oscar.exponents(poly))
    xs = [prod(var_syms .^ e_vec) for e_vec in exp_vecs]
    coeffs' * xs
end

to_symbolic_polynomial.(gens(G))

2-element Vector{Num}:
                    C*D
 -C*D*b + (A^2)*(B^3)*k

## Example: Dimer production

In [187]:
dimer_production = @reaction_network begin
    λₘ, 0 --> mRNA
    λₚ, mRNA --> mRNA + P
    (k₁, k₂), 2P <--> P₂
    δ, (mRNA, P, P₂) --> 0
end

Model ##ReactionSystem#308:
Unknowns (3): see unknowns(##ReactionSystem#308)
  mRNA(t)
  P(t)
  P₂(t)
Parameters (5): see parameters(##ReactionSystem#308)
  λₘ
  λₚ
  k₁
  k₂
  ⋮

In [188]:
odesys = convert(ODESystem, dimer_production, combinatoric_ratelaws=true)

Model ##ReactionSystem#308:
Equations (3):
  3 standard: see equations(##ReactionSystem#308)
Unknowns (3): see unknowns(##ReactionSystem#308)
  mRNA(t)
  P(t)
  P₂(t)
Parameters (5): see parameters(##ReactionSystem#308)
  λₘ
  λₚ
  k₁
  k₂
  ⋮

In [189]:
vars = unknowns(odesys)
params = parameters(odesys)
eqn_rhs = [eq.rhs for eq in equations(odesys)]
I = polynomial_ideal(eqn_rhs, vars, params)

Ideal generated by
  -δ*mRNA + λₘ
  λₚ*mRNA - k₁*P^2 - δ*P + 2*k₂*P₂
  1//2*k₁*P^2 + (-k₂ - δ)*P₂

In [190]:
G = groebner_basis(I)

Gröbner basis with elements
  1: δ^2*P + 2*δ^2*P₂ - λₘ*λₚ
  2: δ*mRNA - λₘ
  3: 4*k₁*δ^2*P₂^2 + λₘ*λₚ*k₁*P + (-2*λₘ*λₚ*k₁ - 2*k₂*δ^2 - 2*δ^3)*P₂
with respect to the ordering
  degrevlex([mRNA, P, P₂])

In [191]:
@assert dim(I) == 0
G = groebner_basis(I, algorithm=:fglm, ordering=lex(base_ring(I)))

Gröbner basis with elements
  1: 4*k₁*δ^4*P₂^2 + (-4*λₘ*λₚ*k₁*δ^2 - 2*k₂*δ^4 - 2*δ^5)*P₂ + λₘ^2*λₚ^2*k₁
  2: δ^2*P + 2*δ^2*P₂ - λₘ*λₚ
  3: δ*mRNA - λₘ
with respect to the ordering
  lex([mRNA, P, P₂])

In [192]:
Oscar.vars.(gens(G))

3-element Vector{Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}}}:
 [P₂]
 [P, P₂]
 [mRNA]

In [193]:
to_symbolic_polynomial.(gens(G))

3-element Vector{Num}:
 k₁*(λₘ^2)*(λₚ^2) + P₂*(-4k₁*(δ^2)*λₘ*λₚ - 2k₂*(δ^4) - 2(δ^5)) + 4(P₂^2)*k₁*(δ^4)
                                                     -λₘ*λₚ + P*(δ^2) + 2P₂*(δ^2)
                                                                     -λₘ + mRNA*δ